# CMIP6 Deltas

This notebook is for downloading monthly CMIP6 data and computing deltas. It is for an ad-hoc project to generate some useful CMIP6 summaries for Alaska while we await the infrastructure upgrades for accessing and working with larger amounts of data. The data downloaded here may be used in other efforts as well, such as statistical downscaling.

# Download monthly CMIP6 data

### Copernicus API

This notebook will use the [Copernicus API](https://cds.climate.copernicus.eu/api-how-to#install-the-cds-api-key) to access data.

The `cdsapi` python package will need to be installed, it is available via `pip` and on conda-forge: `conda install -c conda-forge cdsapi`. 

Credentials need to be placed in `$HOME/.cdsapirc` per the instructions at the link above.

In [34]:
# Setup - define all imports and paths

import logging
import os
from multiprocessing import Pool
from pathlib import Path
from zipfile import ZipFile
import cdsapi
import tqdm
import pandas as pd
# download module
import download


base_dir = Path(os.getenv("BASE_DIR") or "/workspace/Shared/Tech_Projects/cmip6/project_data")
out_dir = Path(os.getenv("OUTPUT_DIR") or "/workspace/Shared/Tech_Projects/cmip6/final_products")
scratch_dir = Path(os.getenv("SCRATCH_DIR") or "/atlas_scratch/kmredilla/cmip6")

anc_dir = base_dir.joinpath("ancillary")
anc_dir.mkdir(exist_ok=True)

# tracking CSV filepath
tracker_fp = anc_dir.joinpath("download_tracker.csv")
    
# make directory for collecting text files for capturing
# the cdsapi logs
download_log_dir = scratch_dir.joinpath("download_logs")
download_log_dir.mkdir(exist_ok=True)

# need main folder for raw data
raw_dir = base_dir.joinpath("raw")
raw_dir.mkdir(exist_ok=True)
# folders for met var data, for land surface data, and water balance data
# meteorological data
met_group_name = "met_data"
met_dir = raw_dir.joinpath(met_group_name)
met_dir.mkdir(exist_ok=True)
# land surface data
land_group_name = "land_data"
land_dir = raw_dir.joinpath(land_group_name)
land_dir.mkdir(exist_ok=True)
# water balance data
wb_group_name = "water_balance_data"
wb_dir = raw_dir.joinpath(wb_group_name)
wb_dir.mkdir(exist_ok=True)

### Targets

This section describes the various subsets of CMIP6 data we are interested in. There are three main groups:
1. meterological data at monthly resolution (temp, precip, SLP)
2. model data, for lack of a better term - data about the models
3. water balance data at monthly resolution

#### Meteorological data
**Temporal resolution**: Monthly  
**Experiment (4)**: Historical, SSP1-2.6, SSP2-4.5, SSP5-8.5  
**Variable (3)**: Near-surface air temperature, Precipitation, Sea level pressure  
**Model (12)**: ACCESS-CM2, CESM2, CNRM-CM6-1-HR, EC-Earth3-Veg-LR, GFDL-ESM4, HadGEM3-GC31-LL, HadGEM3-GC31-MM, KACE-1-0-G, MIROC6, MPI-ESM1-2-LR, MRI-ESM2-0, NorESM2-MM  


#### Land surface data
**Temporal resolution**: Fixed (no temporal resolution)  
**Experiment (1)**: Historical  
**Variable (3)**: Surface altitude, Percentage of the grid cell occupied by land including lakes, Sea area percentage  
**Model (?)**: Subset of above, available  

#### Water balance data
**Temporal resolution**: Monthly  
**Experiment (3)**: Historical, SSP2-4.5, SSP5-8.5  
**Variable (5)**: Evaporation including sublimation and transpiration, Moisture in upper portion of soil column, snowfall flux, surface snow amount, total runoff  
**Model (6)**: CESM2, EC-Earth3-Veg-LR, GFDL-ESM4, MIROC6, MRI-ESM2-0, NorESM2-MM  

#### Fixed parameters
For all of these groups, the following parameters will be constant:  
**Level (1)**: single levels   
**Geographical Area**: Sub-region extraction, North: 74, South 50, West 174, East -120   
**Temporal subset**: Whole available temporal range  


### Notes for downloading

Here are some notes for downloading based on preliminary explorations:  

* the CDS API does not support queries across the antimeridian so there will need to be two bounding boxes queried and then mosaicked together:
    1. (174, 50, 180, 74)
    2. (-180, 50, -120, 74)
* the CDS API does not support multiple values for a given field and so requests should be broken up by model, scenario (experiment), and variable
* the longitude coordiantes for "sea area percentage" seem to be on a strictly positive axis (0, 360) and the positive bbox of the above (74, 174, 50, 240) causes errors, so we might just need to grab the whole grid for these ones.

See the Appendix section for some examples of the above issues.

### Define targets in a tracking table

Make a table for tracking the progress of downloads with the following columns:

* `data_group`: which of the above groups the download is for, either `"met_vars"`, `"land_surface"`, or `"water_balance"`
* `model`
* `scenario`
* `variable`
* `t_res`: temporal resolution, either `"monthly"` or `"fixed"`
* `bbox`: spatial extent, in form (N, W, S, E), should be either `(74, 174, 50, -120)` or `None` (for sea area percentage only)
* `result`: download result, either `"pass"`, `"fail"`, or `None`
* `fail_reason`: fail message where applicable, or `None`
* `zip_path`: path to where downloaded `.zip` data in `$SCRATCH_DIR` should be written
* `base_path`: Path to where data should be unzipped and placed in `$BASE_DIR`

In [24]:
# reset tracker - set to True to reset tracking spreadsheet
reset_tracker = True

import pandas as pd

# make the tracking dataframe
column_names = [
    "data_group",
    "model",
    "scenario",
    "variable",
    "t_res",
    "bbox",
    "hemisphere", # east or west
    "levels",
    "format",
    "result",
    "fail_reason",
    "zip_path",
    "base_path",
    "log_path",
]
df = pd.DataFrame(columns=column_names)

if reset_tracker:
    df.to_csv(tracker_fp, index=False)

Start specifying the options for populating the table.

##### Models:

In [4]:
# initialize a lookup dict for CDS API
api_lu = {}

# full names of models (for reference at this point)
models = [
    "ACCESS-CM2",
    "CESM2",
    "CNRM-CM6-1-HR",
    "EC-Earth3-Veg-LR",
    "GFDL-ESM4",
    "HadGEM3-GC31-LL",
    "HadGEM3-GC31-MM",
    "KACE-1-0-G",
    "MIROC6",
    "MPI-ESM1-2-LR",
    "MRI-ESM2-0",
    "NorESM2-MM",
]
# api names for models are simply lower case and underscores instead of hyphens
api_lu["models"] = {model: model.lower().replace("-", "_") for model in models}

# subset of models for the water balanace data
wb_models = [
    "CESM2",
    "EC-Earth3-Veg-LR",
    "GFDL-ESM4",
    "MIROC6",
    "MRI-ESM2-0",
    "NorESM2-MM",
]

##### Scenarios (experiments):

In [5]:
scenarios = [
    "historical",
    "SSP1-2.6",
    "SSP2-4.5",
    "SSP5-8.5",
]

# water balance scenarios
wb_scenarios = [
    "historical",
    "SSP2-4.5",
    "SSP5-8.5",
]

# lookups for API
api_lu["scenarios"] = {
    "historical": "historical",
    "SSP1-2.6": "ssp1_2_6",
    "SSP2-4.5": "ssp2_4_5",
    "SSP5-8.5": "ssp5_8_5",
}

##### Variables:

In [6]:
met_varnames = ["tas", "pr", "psl"]
land_varnames = ["orog", "sftlf", "sftof"]
wb_varnames = ["evspsbl", "mrsos", "prsn", "snw", "mrro"]

# variable names
api_lu["varnames"] = {
    "tas": "near_surface_air_temperature",
    "pr": "precipitation",
    "psl": "sea_level_pressure",
    "orog": "surface_altitude",
    "sftlf": "percentage_of_the_grid_cell_occupied_by_land_including_lakes",
    "sftof": "sea_area_percentage",
    "evspsbl": "evaporation_including_sublimation_and_transpiration",
    "mrsos": "moisture_in_upper_portion_of_soil_column",
    "prsn": "snowfall_flux",
    "snw": "surface_snow_amount",
    "mrro": "total_runoff",
}

##### Constants:

In [8]:
# these will stay constant over most combinations
# two bboxes, "east" one will be the one in the eastern hemisphere (west of dateline)
bboxes = {"west": [74, -180, 50, -120], "east": [74, 174, 50, 180]}
level = "single_levels"
api_format = "zip"

Start populating the table:

##### Met vars:

In [25]:
# load the df
df = pd.read_csv(tracker_fp)

monthly_zip_fn = "{}_monthly_mean_{}_{}_{}.zip"
for model in models:
    for scenario in scenarios:
        for varname in met_varnames:
            for bbox in bboxes:
                row_df = {
                    "data_group": met_group_name,
                    "model": api_lu["models"][model],
                    "scenario": api_lu["scenarios"][scenario],
                    "variable": api_lu["varnames"][varname],
                    "t_res": "monthly",
                    "bbox": bboxes[bbox],
                    "hemisphere": bbox,
                    "levels": level,
                    "format": api_format,
                    "result": None,
                    "fail_reason": None,
                    "base_path": None,
                    "zip_path": scratch_dir.joinpath(
                        monthly_zip_fn.format(varname, model, scenario, bbox)
                    )
                }
                # make stdout_path for writing printed results of retrieve() function
                row_df["log_path"] = download_log_dir.joinpath(
                    row_df["zip_path"].name.replace(".zip", "_download_log.txt")
                )
                df = df.append(row_df, ignore_index=True)

##### Land surface vars

In [26]:
land_zip_fn = "{}_fx_{}_historical_{}.zip"
for model in models:
    for varname in land_varnames:
        for bbox in bboxes:
            row_df = {
                "data_group": land_group_name,
                "model": api_lu["models"][model],
                "scenario": "historical",
                "variable": api_lu["varnames"][varname],
                "t_res": "fixed",
                "bbox": bboxes[bbox],
                "hemisphere": bbox,
                "levels": level,
                "format": api_format,
                "result": None,
                "fail_reason": None,
                "base_path": None,
                "zip_path": scratch_dir.joinpath(
                    land_zip_fn.format(varname, model, bbox)
                )
            }
            # set the bbox to None if varname is sftof (sea area percentage)
            if varname == "sftof":
                row_df["bbox"] = None
            # make stdout_path for writing printed results of retrieve() function
            row_df["log_path"] = download_log_dir.joinpath(
                row_df["zip_path"].name.replace(".zip", "_download_log.txt")
            )
            df = df.append(row_df, ignore_index=True)

##### Water balance vars

In [27]:
for model in wb_models:
    for scenario in wb_scenarios:
        for varname in wb_varnames:
            for bbox in bboxes:
                row_df = {
                    "data_group": wb_group_name,
                    "model": api_lu["models"][model],
                    "scenario": api_lu["scenarios"][scenario],
                    "variable": api_lu["varnames"][varname],
                    "t_res": "monthly",
                    "bbox": bboxes[bbox],
                    "hemisphere": bbox,
                    "levels": level,
                    "format": api_format,
                    "result": None,
                    "fail_reason": None,
                    "base_path": None,
                    "zip_path": scratch_dir.joinpath(
                        monthly_zip_fn.format(varname, model, scenario, bbox)
                    )
                }
                # make stdout_path for writing printed results of retrieve() function
                row_df["log_path"] = download_log_dir.joinpath(
                    row_df["zip_path"].name.replace(".zip", "_download_log.txt")
                )
                df = df.append(row_df, ignore_index=True)

Reset the index to get a unique ID for each row, and call that column "id":

In [28]:
df.reset_index().rename(columns={"index": "id"}).to_csv(tracker_fp, index=False)

### Run the downloads

Attempt to download all combinations specified in each row in the tracker table via the CDS API. Do this in parallel. 

In [35]:
# load and update the tracker before trying to download anything
df = download.update_tracker(tracker_fp, raw_dir)
rows = [row[1].copy(deep=True) for row in df.iterrows()]

with Pool(32) as pool:
    new_rows = [
        result for result in tqdm.tqdm(
            pool.imap_unordered(download.download, rows), total=len(rows))
    ]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 540/540 [1:14:58<00:00,  8.33s/it]


Write results of downloading:

In [83]:
df = pd.DataFrame(new_rows).sort_values("id")
df.to_csv(tracker_fp, index=False)

Update the tracker after downloading:

In [121]:
df = download.update_tracker(tracker_fp, raw_dir)

Unzip files in batch and update tracker as this happens:

In [123]:
df = download.batch_unzip(tracker_fp, raw_dir)

1 data files extracted from zips


How many data files were successfully downloaded and extracted:

In [124]:
print(f"Meteorological data: {len(list(met_dir.glob('*.nc')))} files")
print(f"Land surface data: {len(list(land_dir.glob('*.nc')))} files")
print(f"Water balance data: {len(list(wb_dir.glob('*.nc')))} files")

Meteorological data: 280 files
Land surface data: 36 files
Water balance data: 164 files


How many files would not unzip?

In [94]:
temp_df = df[df["fail_reason"] == "BadZipFile"]
print(f"{len(temp_df)} failed extractions:")
temp_df[["data_group", "model", "scenario", "variable"]]

0 failed extractions:


,data_group,model,scenario,variable


How many files failed to download?

In [95]:
temp_df = df[df["result"] == "fail"]
print(f"{len(temp_df)} failed downloads:")
temp_df[["data_group", "model", "scenario", "variable", "hemisphere"]]

61 failed downloads:


,data_group,model,scenario,variable,hemisphere
72,met_data,ec_earth3_veg_lr,historical,near_surface_air_temperature,west
73,met_data,ec_earth3_veg_lr,historical,near_surface_air_temperature,east
156,met_data,hadgem3_gc31_mm,ssp2_4_5,near_surface_air_temperature,west
157,met_data,hadgem3_gc31_mm,ssp2_4_5,near_surface_air_temperature,east
158,met_data,hadgem3_gc31_mm,ssp2_4_5,precipitation,west
...,...,...,...,...,...
429,water_balance_data,gfdl_esm4,historical,total_runoff,east
436,water_balance_data,gfdl_esm4,ssp2_4_5,surface_snow_amount,west
437,water_balance_data,gfdl_esm4,ssp2_4_5,surface_snow_amount,east
446,water_balance_data,gfdl_esm4,ssp5_8_5,surface_snow_amount,west


## Appendix

This section is just for showing some things to look out for when downloading data.

### Longitude restrictions for "sea area percentage"

the longitude coordiantes for "sea area percentage" seem to be on a strictly positive axis (0, 360) and the positive bbox of the above (74, 174, 50, 240) causes errors, so we might just need to grab the whole grid for these ones.

##### bbox with negative value:

In [76]:
fp = scratch_dir.joinpath("test.zip")

c = cdsapi.Client()

try:
    c.retrieve(
        'projections-cmip6',
        {
            'format': api_format,
            'temporal_resolution': "fixed",
            'experiment': "historical",
            'level': 'single_levels',
            'variable': "sea_area_percentage",
            'model': "noresm2_lm",
            "area": [74, 174, 50, -120]
        },
        fp
    )
except Exception as exc:
    print(exc.args)

2022-02-14 13:29:11,313 INFO Welcome to the CDS
2022-02-14 13:29:11,314 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2022-02-14 13:29:11,501 INFO Request is queued
2022-02-14 13:29:12,681 INFO Request is running
2022-02-14 13:29:16,795 INFO Request is failed
2022-02-14 13:29:16,796 ERROR Message: an internal error occurred processing your request
2022-02-14 13:29:16,796 ERROR Reason:  Process error: The requested longitude subset -120.0, 174.0 is not within the longitude bounds of this dataset and the data could not be converted to this longitude frame successfully. Please re-run your request with longitudes within the bounds of the dataset: 0.00, 359.99
2022-02-14 13:29:16,797 ERROR   Traceback (most recent call last):
2022-02-14 13:29:16,798 ERROR     File "/usr/local/lib/python3.6/site-packages/rooki/results.py", line 33, in url
2022-02-14 13:29:16,799 ERROR       return self.response.get()[0]
2022-02-14 13:29:16,800 ERROR     File "/u

('an internal error occurred processing your request. Process error: The requested longitude subset -120.0, 174.0 is not within the longitude bounds of this dataset and the data could not be converted to this longitude frame successfully. Please re-run your request with longitudes within the bounds of the dataset: 0.00, 359.99.',)


##### bbox with corresponding positive value:

In [77]:
fp = scratch_dir.joinpath("test.zip")

c = cdsapi.Client()

try:
    c.retrieve(
        'projections-cmip6',
        {
            'format': api_format,
            'temporal_resolution': "fixed",
            'experiment': "historical",
            'level': 'single_levels',
            'variable': "sea_area_percentage",
            'model': "noresm2_lm",
            "area": [74, 174, 50, 240]
        },
        fp
    )
except Exception as exc:
    print(exc.args)

2022-02-14 13:31:50,746 INFO Welcome to the CDS
2022-02-14 13:31:50,747 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2022-02-14 13:31:50,935 INFO Request is queued
2022-02-14 13:31:52,115 INFO Request is running
2022-02-14 13:31:56,228 INFO Request is failed
2022-02-14 13:31:56,229 ERROR Message: an internal error occurred processing your request
2022-02-14 13:31:56,229 ERROR Reason:  execution failed
2022-02-14 13:31:56,230 ERROR   Traceback (most recent call last):
2022-02-14 13:31:56,231 ERROR     File "/usr/local/lib/python3.6/site-packages/birdy/client/base.py", line 347, in _execute
2022-02-14 13:31:56,232 ERROR       pid, inputs=wps_inputs, output=wps_outputs, mode=mode
2022-02-14 13:31:56,233 ERROR     File "/usr/local/lib/python3.6/site-packages/owslib/wps.py", line 359, in execute
2022-02-14 13:31:56,234 ERROR       response = execution.submitRequest(request)
2022-02-14 13:31:56,234 ERROR     File "/usr/local/lib/python3.6/site-

('an internal error occurred processing your request. execution failed.',)


### No support for multiple field values

The CDS API does not support multiple values for a given field:

In [79]:
# example with two variables supplied
fp = scratch_dir.joinpath("test.zip")

c = cdsapi.Client()

try:
    c.retrieve(
        'projections-cmip6',
        {
            'format': api_format,
            'temporal_resolution': "monthly",
            'experiment': "historical",
            'level': 'single_levels',
            'variable': ["percipitation", "near_surface_air_temperature"],
            'model': "cesm2",
            "area": [74, 174, 50, -120],
            "date": "1850-01-15/1851-12-31",
        },
        fp
    )
except Exception as exc:
    print(exc.args)

2022-02-14 13:36:23,676 INFO Welcome to the CDS
2022-02-14 13:36:23,677 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2022-02-14 13:36:23,866 INFO Request is queued
2022-02-14 13:36:25,049 INFO Request is running
2022-02-14 13:36:29,161 INFO Request is failed
2022-02-14 13:36:29,162 ERROR Message: an internal error occurred processing your request
2022-02-14 13:36:29,163 ERROR Reason:  CMIP6 requests are limited to a single variable per request; please split your request into multiple requests.
2022-02-14 13:36:29,164 ERROR   Traceback (most recent call last):
2022-02-14 13:36:29,165 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/cdshandlers/services/handler.py", line 55, in handle_request
2022-02-14 13:36:29,166 ERROR       result = cached(context.method, proc, context, context.args, context.kwargs)
2022-02-14 13:36:29,167 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/caching.py", line 108, in cached
2022-02-14 13:36:29,168

('an internal error occurred processing your request. CMIP6 requests are limited to a single variable per request; please split your request into multiple requests..',)
